In [23]:
import cv2
import numpy as np
import math
import time

In [24]:
def dist(refimg, frameimg, m, n, i,j):
    refimg = np.array(refimg).astype(np.int64)
    frameimg = np.array(frameimg)[i: i + m, j: j + n].astype(np.int64)
    return np.sum((frameimg - refimg)**2)

In [25]:
def check(X,Y,x,y):
    return x>=0 and x<X and y>=0 and y<Y

In [26]:
def drawbox(frame,M,N,min_i,min_j):
    for i in range(min_i - 1, min_i + M + 1):
        frame[i][min_j - 1][0] = frame[i][min_j + N][0] = np.int8(0)
        frame[i][min_j - 1][1] = frame[i][min_j + N][1] = np.int8(0)
        frame[i][min_j - 1][2] = frame[i][min_j + N][2] = np.int8(255)

    for j in range(min_j - 1, min_j + N + 1):
        frame[min_i - 1][j][0] = frame[min_i + M][j][0] = np.int8(0)
        frame[min_i - 1][j][1] = frame[min_i + M][j][1] = np.int8(0)
        frame[min_i - 1][j][2] = frame[min_i + M][j][2] = np.int8(255)
    return frame

In [27]:
def videopublisher(frames,p,frame_rate,I,J,path):
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(path+ str(p) + '.avi', fourcc, frame_rate, (J, I), True)

    for frame in frames:
        out.write(frame)
    out.release()

    print("Video saved!")

In [28]:
def exhaustive_search(refimg, frames, frame_rate, p):


    out_frames = []
    prev_x = prev_y = 0
    co = 0
    for frame in frames:
        d_min = math.inf
        min_i = min_j = -1
        M,N = refimg.shape
        I,J = frame.shape
        if co == 0:
            for i in range(I - M):
                for j in range(J - N):
                    d = dist(refimg, frame, M, N, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j
            co+=1
        else:
            for i in range(prev_x - p, prev_x + p + 1):
                for j in range(prev_y - p, prev_y + p + 1):
                    if not check(I-M,J-N,i,j):
                        continue
                    d = dist(refimg, frame, M, N, i, j)
                    co+=1
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j

        rgbframe = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        rgbframe = drawbox(rgbframe,M,N,min_i,min_j)
        out_frames.append(rgbframe)

    videopublisher(out_frames,p,frame_rate,I,J,'Output_Videos/exhaust_')
    
    return co/(1.0*len(frames))

In [29]:

def logarithmic_search(refimg, frames, frame_rate, p):
    out_frames = []
    prev_x = prev_y = 0
    co = 0

    for frame in frames:
        d_min = math.inf
        min_i = min_j = -1
        M,N = refimg.shape
        I,J = frame.shape
        if co == 0:
            for i in range(I - M):
                for j in range(J - N):
                    d = dist(refimg, frame, M, N, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j
            co+=1
        else:
            spacing = round(2**(math.ceil(math.log(p, 2)) - 1))
            while spacing >= 1:
                d_min = math.inf
                min_i = min_j = -1
                for i in [prev_x - spacing, prev_x, prev_x + spacing]:
                    for j in [prev_y - spacing, prev_y, prev_y + spacing]:
                        if not check(I-M,J-N,i,j):
                            continue
                        d = dist(refimg, frame, M, N, i, j)
                        co+=1
                        if d < d_min:
                            d_min = d
                            min_i = i
                            min_j = j
                prev_x = min_i
                prev_y = min_j
                spacing = round(spacing / 2.0)

        rgbframe = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        rgbframe = drawbox(rgbframe,M,N,min_i,min_j)
        out_frames.append(rgbframe)

    videopublisher(out_frames,p,frame_rate,I,J,'Output_Videos/log_')

    return co/(1.0*len(frames))

In [30]:
def hierarchical_search(refimg, frames, frame_rate, p):
    

    out_frames = []
    prev_x = prev_y = 0
    co = 0

    for frame in frames:
        d_min = math.inf
        min_i = min_j = -1
        M,N = refimg.shape
        I,J = frame.shape
        if co == 0:
            for i in range(I - M):
                for j in range(J - N):
                    d = dist(refimg, frame, M, N, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j
            co+=1
        else:

            p_= round(p/4.0)
            
            spacing = round(2**(math.ceil(math.log(p_, 2)) - 1))
            center_x = math.floor(prev_x/4.0)
            center_y = math.floor(prev_y/4.0)
            cur_frame = cv2.pyrDown(cv2.pyrDown(frame))
            cur_ref = cv2.pyrDown(cv2.pyrDown(refimg))

            while spacing >= 1:
                d_min = math.inf
                min_i = min_j = -1
                for i in [center_x - spacing, center_x, center_x + spacing]:
                    for j in [center_y - spacing, center_y, center_y + spacing]:
                        I_,J_ = cur_frame.shape
                        M_,N_ = cur_ref.shape 
                        if not check(I_-M_,J_-N_,i,j):
                            continue
                        d = dist(cur_ref, cur_frame, M_,N_, i, j)
                        co+=1
                        if d < d_min:
                            d_min = d
                            min_i = i
                            min_j = j
                center_x = min_i
                center_y = min_j
                spacing = round(spacing / 2.0)

            d_min = math.inf
            p_ = 1
            center_x = 2*min_i
            center_y = 2*min_j
            cur_frame = cv2.pyrDown(frame)
            cur_ref = cv2.pyrDown(refimg)

            for i in [center_x - p_,center_x, center_x + p_]:
                for j in [center_y - p_,center_y, center_y + p_ ]:
                    I_,J_ = cur_frame.shape
                    M_,N_ = cur_ref.shape
                    if not check(I_-M_,J_-N_,i,j):
                        continue
                    d = dist(cur_ref, cur_frame, M_,N_, i, j)
                    co+=1
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j

            d_min = math.inf
            p_ = 1
            center_x = 2*min_i
            center_y = 2*min_j
            cur_frame = frame
            cur_ref = refimg
            for i in [center_x - p_,center_x, center_x + p_]:
                for j in [center_y - p_,center_y, center_y + p_ ]:
                    I_,J_ = cur_frame.shape
                    M_,N_ = cur_ref.shape
                    if not check(I_-M_,J_-N_,i,j):
                        continue
                    d = dist(cur_ref, cur_frame, M_,N_, i, j)
                    co+=1
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j

        rgbframe = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        rgbframe = drawbox(rgbframe,M,N,min_i,min_j)
        out_frames.append(rgbframe)

    videopublisher(out_frames,p,frame_rate,I,J,'Output_Videos/hierarchy_')

    return co/(1.0*len(frames))

In [31]:
refimg = cv2.imread('reference.jpg')
refimg = cv2.cvtColor(refimg, cv2.COLOR_BGR2GRAY)


In [32]:
inputvideo = cv2.VideoCapture('input.mov')
frames = []
while True:
    ret, frame = inputvideo.read()
    if not ret:
        break
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
fps = inputvideo.get(cv2.CAP_PROP_FPS)
inputvideo.release()

In [33]:
p = 8
while p<=8:
    c1 = exhaustive_search(refimg, frames, fps, p)
    c2 = logarithmic_search(refimg, frames, fps, p)
    c3 = hierarchical_search(refimg, frames, fps, p)
    print(p," ",c1," ",c2," ",c3)
    p*=2

Video saved!
Video saved!
Video saved!
8   288.62105263157895   26.96578947368421   26.96578947368421
